In [1]:
!pip install tensorflow-gpu==1.15
# @title Preparation
!pip install -q keras-bert keras-rectified-adam
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     |████████████████████████████████| 411.5MB 38kB/s 
     |████████████████████████████████| 3.8MB 23.4MB/s 
     |████████████████████████████████| 512kB 42.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3a69c056a2e3a82d79e92adeb0dcea582cb936ff8dd2d530be621e3b68bf0af9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc4 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uni

In [2]:
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras_bert import get_custom_objects
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
import pandas as pd
import tensorflow.keras.backend as K
import sys
from sklearn.metrics import classification_report
from google.colab import drive

Using TensorFlow backend.


In [0]:
# @title Constants

np.random.seed(42)
SEQ_LEN = 128
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5

In [0]:
# @title Environment
import os
pretrained_path = '/content/drive/My Drive/codiesp/'
config_path = os.path.join(pretrained_path, 'config.json')
checkpoint_path = os.path.join(pretrained_path, 'model.ckpt-2000000')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [5]:
# @title Load Basic Model
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 10.72 MiB/s, done.
Resolving deltas: 100% (185/185), done.



In [6]:
# @title Load Data
!pip install category_encoders==1.3.0
import joblib
import pandas as pd
from keras import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import random
from keras.layers import Input
import keras
from keras.layers import Conv1D , Embedding
from keras.layers import Dropout
from keras.layers import MaxPool1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import GlobalMaxPool1D
from keras.layers import Bidirectional
import category_encoders as ce

from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

def remove_symbol(s):
    s = s.replace(",", "")
    s = s.replace(".", "")
    s = s.replace(";", "")
    s = s.replace(":", "")
    s = s.replace("_", "")
    s = s.replace("+", "")
    s = s.replace("ª", "")
    s = s.replace("-", "")
    s = s.replace("<", "")
    s = s.replace(">", "")
    s = s.replace("!", "")
    s = s.replace("?", "")
    s = s.replace("(", "")
    s = s.replace(")", "")
    s = s.replace("[", "")
    s = s.replace("]", "")
    s = s.replace("'", "")
    s = s.replace("0", "")
    s = s.replace("1", "")
    s = s.replace("2", "")
    s = s.replace("3", "")
    s = s.replace("4", "")
    s = s.replace("5", "")
    s = s.replace("6", "")
    s = s.replace("7", "")
    s = s.replace("8", "")
    s = s.replace("9", "")
    s = s.replace("%", "")
    s = s.strip()
    s = s.lower()
    return s

# Rimozione delle stopword
def clar_text(text):

    t = remove_symbol(str(text).strip().lower())
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))

    tt = ""
    for it in tokens:
      tt = tt +" "+it
    return tt

def _pad(input_ids, max_seq_len):
    x = []
    input_ids = input_ids[:min(len(input_ids), max_seq_len - 2)]
    input_ids = input_ids + [0] * (max_seq_len - len(input_ids))
    return np.array(input_ids)

#LOADING DATASET
df = pd.read_csv('/content/drive/My Drive/codiesp/Train_with_emptyclass.csv')
df = df[['Code', 'Desc']]
# df = df[pd.notnull(df['desc'])]
#print(df.head(10))
#print(df.shape)

df.index = range(df.shape[0])
print("Parole: " + str(df['Desc'].apply(lambda x: len(x.split(' '))).sum()))  # ci sono circa 211456 parole

# rimozione SOLO dei simboli (nessuno stemming e nessuna rimozione delle stopword)
df['Desc'] = df['Desc'].apply(remove_symbol)
#print(df.head(10))

# Acquisizione delle stop word
file_stopw = open("/content/drive/My Drive/codiesp/stop_word.pck", "rb")
stop_word = pickle.load(file_stopw)
df['Desc'] = df['Desc'].apply(clar_text)

#suddivisione train_test
train = df

df1 = pd.read_csv('/content/drive/My Drive/codiesp/TestSet.csv')
df1 = df1[['id', 'Desc']]
# rimozione SOLO dei simboli (nessuno stemming e nessuna rimozione delle stopword)
df1['Desc'] = df1['Desc'].apply(remove_symbol)
df1['Desc'] = df1['Desc'].apply(clar_text)
test = df1

#prepare class encoder
le = ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
labels = le.fit(list(df['Code']))
#mapa = [0,1]
mapa = le.category_mapping[0]['mapping']
labels_map = []
i = 0
for a,b in mapa:
    labels_map.append(a)
print(labels_map)

#Tokenization
#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(vocab_path, do_lower_case=True)
indices_train = []
indices_test = []

for text in train['Desc']:
  tk = tokenizer.tokenize(text)
  tokens = ["[CLS]"] + tk + ["[SEP]"]
  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  token_ids = _pad(token_ids,SEQ_LEN)
  indices_train.append(token_ids)

for text in test['Desc']:
  tk = tokenizer.tokenize(text)
  tokens = ["[CLS]"] + tk + ["[SEP]"]
  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  token_ids = _pad(token_ids,SEQ_LEN)
  indices_test.append(token_ids)

indices_train = [indices_train, np.zeros_like(indices_train)]
indices_test= [indices_test, np.zeros_like(indices_test)]

train_labels = train['Code']
train_labes_indexes = []
for label in train_labels:
  train_labes_indexes.append(labels_map.index(label))

test_ids = test['id']
#test_labes_indexes = []
#for label in test_labels:
#  test_labes_indexes.append(labels_map.index(label))

#print(test_labels)

     |████████████████████████████████| 61kB 2.6MB/s 
Parole: 1600976
['z20.818', 'm25.50', 'a23.9', 'i87.8', 'e04.9', 'r50.9', 'n44.8', 'r60.9', 'n45.3', 'r52', 'i83.90', 'emp', 'r58', 'd30.3', 'd49.59', 'n50.9', 'r63.4', 'r19.00', 'n50.89', 'c63.10', 'k74.60', 'g82.20', 'i85.00', 'n32.9', 'r20.1', 'z96.0', 'g04.1', 'c22.0', 'k76.6', 'k70.30', 'z98.85', 'i85.01', 'r74.0', 'b33.3', 'r53.1', 'i10', 'r53.81', 'g71.8', 'g82.21', 'r18.8', 'n32.89', 'n32.0', 'k40.90', 'n13.30', 'n28.89', 'c78.7', 'c77.9', 'k56.60', 'n13.9', 'n13.8', 'r59.9', 'k31.7', 'c64.9', 'c16.9', 'n28.9', 'r18.0', 'c80.0', 'k59.00', 'd64.9', 'q61.3', 'd50.9', 'r63.0', 'r10.2', 'n80.9', 'n85.8', 'k44.9', 'r31.9', 'k66.0', 'n23', 'n80.8', 'n26.1', 'z90.710', 'c66.9', 'n28.1', 'D49.511', 'd30.01', 'd49.511', 'i82.90', 'c65.9', 'n20.0', 'r10.30', 'r36.1', 'n41.0', 'i86.1', 'n44.2', 'd18.00', 'd40.11', 'n53.12', 'j43.9', 'c34.91', 'r31.29', 'r23.1', 'c79.02', 'j44.9', 'c34.90', 'c79.00', 'r59.0', 'd49.1', 'r69', 'r10.9', 'c

In [7]:
len(labels_map)

1789

In [0]:
!pip install joblib
import joblib

labels_map = joblib.load('/content/drive/My Drive/codiesp/labels_map_bert_only_01.joblib')

['/content/drive/My Drive/codiesp/labels_map_bert_only_01.joblib']

In [8]:
bert = load_trained_model_from_checkpoint(
    config_file=config_path,
    checkpoint_file=checkpoint_path,
    training=True,
    trainable=True,
    seq_len=128
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# @title Build Custom Model

inputs = bert.inputs[:2]
dense = bert.get_layer('NSP-Dense').output
dense1 = keras.layers.Dense(units=1000, activation='tanh') (dense)
outputs = keras.layers.Dense(units=len(labels_map), activation='softmax')(dense1)

modelk = keras.models.Model(inputs, outputs)
modelk.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [10]:
# @title Initialize Variables
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# @title Fit

filepath="/content/drive/My Drive/codiesp/bert_with_empty.{epoch:05d}-{val_loss:.5f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks_list = [
    checkpoint
]

modelk.fit(
    indices_train,
    train_labes_indexes,
    epochs=20,
    batch_size=32,
    validation_split = 0.10,
    callbacks=callbacks_list
)

Train on 44369 samples, validate on 4930 samples
Epoch 1/20
44369/44369 [==============================] - 2370s 53ms/step - loss: 1.6051 - sparse_categorical_accuracy: 0.8366 - val_loss: 3.9107 - val_sparse_categorical_accuracy: 0.5282

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.52819, saving model to /content/drive/My Drive/codiesp/bert_with_empty.00001-3.91068.hdf5
Epoch 2/20
44369/44369 [==============================] - 2366s 53ms/step - loss: 1.0435 - sparse_categorical_accuracy: 0.8610 - val_loss: 3.9061 - val_sparse_categorical_accuracy: 0.5282

Epoch 00002: val_sparse_categorical_accuracy did not improve from 0.52819
Epoch 3/20
44369/44369 [==============================] - 2371s 53ms/step - loss: 0.9991 - sparse_categorical_accuracy: 0.8612 - val_loss: 4.0099 - val_sparse_categorical_accuracy: 0.5282

Epoch 00003: val_sparse_categorical_accuracy did not improve from 0.52819
Epoch 4/20
44369/44369 [==============================] - 2379s 54ms/step - l

In [0]:
#modelk.save('/content/drive/My Drive/codiesp/final_model_only_01.h5')
modelk.load_weights('/content/drive/My Drive/codiesp/bert_with_empty.00008-3.98479.hdf5')

In [12]:
# @title Predict
predicts = modelk.predict(indices_test, verbose=True)

 1408/53220 [..............................] - ETA: 7:01:20

KeyboardInterrupt: ignored

In [0]:
res_encoded = []
for a in predicts:
    val = a.argmax()
    res_encoded.append(labels_map[val])

for i in range(0,len(res_encoded)):
  label = res_encoded[i]
  if label != 

#print(res_encoded)
#res_labels = le.inverse_transform(DataFrame.res_encoded)

#print('Testing accuracy %s' % accuracy_score(test_labels, res_encoded))
#print('Testing F1 score: {}'.format(f1_score(test_labels, res_encoded, average='weighted')))


In [0]:
extracted_res = []
extractede_ids =[]
j =0
previous_id = test_ids[0]

for result in predicts:
  id = test_ids[j]

  if id != previous_id:
    od = OrderedDict(max_vals) 
    #print(str(id)+' '+str(od))

    for key, value in od.items():
      extracted_res.append(value.lower())
      extractede_ids.append(previous_id)
    max_vals = {}
    previous_id = id

  i = 0
  for a in result:
    if a > 0.10:
      if labels_map[i] != 'emp':
        max_vals[str(a)] = labels_map[i]
    i = i+1
  if len(max_vals) == 0:
    v = result.max()
    index = result.argmax()
    max_vals[str(v)] = labels_map[index]

  j = j+1

#Last item
od = OrderedDict(max_vals) 

for key, value in od.items():
    extracted_res.append(value.lower())
    extractede_ids.append(previous_id)

In [0]:
file = open('task1_run3_bert_multiclass.tsv','w+')

for i in range(0, len(extracted_res)):
  file.write(str(extractede_ids[i])+'\t'+str(extracted_res[i])+'\n')
  file.flush()

file.close()

In [0]:
!pip install scikit-learn
from sklearn.metrics import classification_report


In [0]:
print(classification_report(test_labels, res_encoded,digits=5))